<a href="https://colab.research.google.com/github/rajeshmore1/Prompt-Engineering-With-Llama3/blob/main/finetune_llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U transformers datasets accelerate peft trl bitsandbytes wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━

In [2]:
import wandb
wandb.login()  # Follow the instructions to enter your API key

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
#a069ee295ae53318d9c7168e13886e52935a649a

import transformers
import datasets
import torch
import wandb

print("Transformers version:", transformers.__version__)
print("Datasets version:", datasets.__version__)
print("PyTorch version:", torch.__version__)
print("WandB version:", wandb.__version__)

Transformers version: 4.41.0
Datasets version: 2.19.1
PyTorch version: 2.2.1+cu121
WandB version: 0.17.0


In [4]:
import wandb
# This command will prompt you to log in if you're not already authenticated
wandb.login()

wandb: Currently logged in as: rajeshmore1 (deccanai). Use `wandb login --relogin` to force relogin


True

In [5]:
import os

# Example of setting an environment variable for API access
os.environ['API_KEY'] = 'a069ee295ae53318d9c7168e13886e52935a649a'

# If you're using Google Colab, this might include setting up paths or configurations specific to Colab's environment
os.environ['COLAB_ENV'] = 'true' if 'google.colab' in str(get_ipython()) else 'false'

In [6]:
os.makedirs('models', exist_ok=True)
os.makedirs('output', exist_ok=True)
os.makedirs('logs', exist_ok=True)
print("Workspace directories are set up.")

Workspace directories are set up.


In [7]:
# Initialize a WandB run
wandb.init(project='llama3-finetuning', entity='rajeshmore1')

# Log a test data point
wandb.log({'test': 1})

# Finish the run
wandb.finish()
print("W&B setup test completed successfully.")

wandb: Currently logged in as: rajeshmore1. Use `wandb login --relogin` to force relogin


test,▁
test,1


W&B setup test completed successfully.


In [8]:
from datasets import load_dataset

# Load the dataset from Hugging Face Hub
dataset = load_dataset("Amod/mental_health_counseling_conversations")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})

In [11]:
print("Dataset structure:", dataset)
print("Sample entry:", dataset['train'][0])

Dataset structure: DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})
Sample entry: {'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?", 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purp

In [12]:
dataset.dtype

AttributeError: 'DatasetDict' object has no attribute 'dtype'

In [15]:
dataset.save_to_disk("dataset.csv")

Saving the dataset (0/1 shards):   0%|          | 0/3512 [00:00<?, ? examples/s]

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

def preprocess_data(example):
    # Tokenize the conversations
    return tokenizer(example['conversation'], truncation=True, padding="max_length", max_length=512)

# Apply the preprocessing function to all entries in the dataset
dataset = dataset.map(preprocess_data, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B.
401 Client Error. (Request ID: Root=1-6649f70d-1acc47f8563596a579f8242c;ae07e8d6-360a-4b36-9e25-e48f2783c172)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted. You must be authenticated to access it.